<h3>Chatbot using Neural Networks</h3>

In this notebook, we are going to use a feed-forward neural network to train a chatbot. This is my personal favourite method of creating a chatbot because we use json files as training data. In addition, the use of Tkinter(GUI) enhances the user experience.

The use of json (JavaScript Oriented Notation) fomart, makes it possible for the chatbot to give different responses to a similar question. The previous NLP chatbot can still do this but less effectively.

Natural Language Processing tools will still be used in this chatbot such as WordNetLemmatizer to analyze, clean and stucture the text data.

Let's go.

In [1]:
# Import required libraries
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
#from keras.optimizers import SGD
from tensorflow.keras.optimizers import SGD
from keras.optimizers import gradient_descent_v2
sgd = gradient_descent_v2.SGD(...)
import random

[nltk_data] Downloading package punkt to /home/alpha/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/alpha/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Create empty lists to hold the text data and another list with words to ignore. Then load the training text data in json fomart.

In [2]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('Alpha.json').read()
intents = json.loads(data_file)

Create a control structure to loop through the file, tokenize each word and append to the empty lists we created. Tokenization has been explained in detail in the previous Chatbot.

In [3]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [4]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

26 documents
6 classes ['age', 'goodbye', 'greeting', 'hobbies', 'jokes', 'name']
40 unique lemmatized words ['a', 'age', 'am', 'anyone', 'are', 'call', 'chris', 'cya', 'day', 'do', 'doing', 'enjoy', 'funny', 'good', 'goodbye', 'have', 'hello', 'hi', 'how', 'i', 'interest', 'is', 'joke', 'later', 'laugh', 'leaving', 'like', 'make', 'me', 'name', 'old', 'see', 'should', 'tell', 'there', 'up', 'what', 'whats', 'you', 'your']


Create the training data

In [5]:
# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


<ipython-input-5-1a891a7a8859>:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


Let's create our feed-forward neural network. It will be a Sequential model with 3 layers. Two layers will have the relu activation function while the third one will have a softmax activation function.
To learn more about activation function, follow this link https://machinelearningmastery.com/choose-an-activation-function-for-deep-learning/.

Two dropout layers will aslo be added. Dropout layers are used to set aside a percentage of neurons to actually 'learn'.
Take note of how the input and output layers are the lengths of train and test data respectively.

We will an optimizer called Stochastic gradient descent with Nesterov accelerated gradient which gives good results for this model.

Our hyperparameters include: 50 epochs and a batch size of 5. You can change this values as well as add more layers with more/less neurons to alter the performance/accuracy of the model.

Lastly, we will save the model.

In [6]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=50, batch_size=5, verbose=1)
model.save('chrisbot_model.h5', hist)

print("model created")

Epoch 1/50


/home/alpha/anaconda3/envs/ICE/lib/python3.8/site-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


6/6 [==============================] - 0s 2ms/step - loss: 1.8649 - accuracy: 0.1154
Epoch 2/50
6/6 [==============================] - 0s 2ms/step - loss: 1.7640 - accuracy: 0.2692
Epoch 3/50
6/6 [==============================] - 0s 2ms/step - loss: 1.7253 - accuracy: 0.2692
Epoch 4/50
6/6 [==============================] - 0s 2ms/step - loss: 1.6642 - accuracy: 0.3462
Epoch 5/50
6/6 [==============================] - 0s 2ms/step - loss: 1.5484 - accuracy: 0.6154
Epoch 6/50
6/6 [==============================] - 0s 2ms/step - loss: 1.4940 - accuracy: 0.6154
Epoch 7/50
6/6 [==============================] - 0s 2ms/step - loss: 1.4917 - accuracy: 0.5385
Epoch 8/50
6/6 [==============================] - 0s 2ms/step - loss: 1.3780 - accuracy: 0.6538
Epoch 9/50
6/6 [==============================] - 0s 2ms/step - loss: 1.2780 - accuracy: 0.6923
Epoch 10/50
6/6 [==============================] - 0s 2ms/step - loss: 1.1861 - accuracy: 0.7692
Epoch 11/50
6/6 [==============================] -

Let's load the model and test it.

In [7]:
from keras.models import load_model
model = load_model('chrisbot_model.h5')
import json
import random
intents = json.loads(open('Alpha.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

In [8]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(msg):
    ints = predict_class(msg, model)
    res = getResponse(ints, intents)
    return res

Let's design our User Interface. We will use Tkinter for a Graphical User Interface

In [9]:
#Creating GUI with tkinter
import tkinter
from tkinter import *


def send():
    msg = EntryBox.get("1.0",'end-1c').strip()
    EntryBox.delete("0.0",END)

    if msg != '':
        ChatLog.config(state=NORMAL)
        ChatLog.insert(END, "You: " + msg + '\n\n')
        ChatLog.config(foreground="#442265", font=("Verdana", 12 ))

        res = chatbot_response(msg)
        ChatLog.insert(END, "Chris: " + res + '\n\n')

        ChatLog.config(state=DISABLED)
        ChatLog.yview(END)


base = Tk()
base.title("Christopher")
base.geometry("400x500")
base.resizable(width=FALSE, height=FALSE)

#Create Chat window
ChatLog = Text(base, bd=0, bg="white", height="8", width="50", font="Arial",)

ChatLog.config(state=DISABLED)

#Bind scrollbar to Chat window
scrollbar = Scrollbar(base, command=ChatLog.yview, cursor="heart")
ChatLog['yscrollcommand'] = scrollbar.set

#Create Button to send message
SendButton = Button(base, font=("Verdana",12,'bold'), text="Send", width="12", height=5,
                    bd=0, bg="#32de97", activebackground="#3c9d9b",fg='#ffffff',
                    command= send )

#Create the box to enter message
EntryBox = Text(base, bd=0, bg="white",width="29", height="5", font="Arial")
#EntryBox.bind("<Return>", send)


#Place all components on the screen
scrollbar.place(x=376,y=6, height=386)
ChatLog.place(x=6,y=6, height=386, width=370)
EntryBox.place(x=128, y=401, height=90, width=265)
SendButton.place(x=6, y=401, height=90)

base.mainloop()